* 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
* 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
* 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
* 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)

examples = [
    {
        "question": "탑건",
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "question": "대부",
        "answer": "👨‍👨‍👦🔫🍝"
    }
]

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a 이모티콘 영화 추천 챗봇, 영화제목을 입력하면 영화를 나타내는 **세 개의 이모티콘**으로 응답해야 합니다. (예: '탑건' -> '🛩️👨‍✈️🔥'. '대부' -> '👨‍👨‍👦🔫🍝')"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=prompt,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

In [2]:
invoke_chain("토토로")

content='👧🏻🐉🌸'


In [3]:
invoke_chain("도라에몽")

content='🧒🏻🔵🎒'


In [4]:
invoke_chain("아바타")

content='🌿🌊🔥'


In [5]:
invoke_chain("지금까지 이야기했던 모든 영화")

content='👧🏻🐉🌸  \n🧒🏻🔵🎒  \n🌿🌊🔥'
